In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


### Create the dataframe for the coordinates of all boroughs in Toronto and create a map of them

In [2]:
# Load the wikipedia page and extract the neighborhood list 
from urllib.request import urlopen
resp = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read().decode("utf-8")

from bs4 import BeautifulSoup
soup = BeautifulSoup(resp,"lxml")
table = soup.find('table',{'class':'wikitable sortable'})
cells = []
for row in table.findAll('tr'):
    if len(row.findAll('td'))>0:
        current_row = []
        for cell in row.findAll('td'):
            current_row.append(cell.find(text=True).strip())
        # If no neighborhood name is assigned to a borough, use borough name as neighborhood name
        if current_row[1]!='Not assigned' and current_row[2]=='Not assigned':
            current_row[2] = current_row[1]
        if current_row[1]!='Not assigned' and current_row[2]!='Not assigned':
            cells.append(current_row)

In [3]:
# Create dataframe
column_names = ["PostalCode","Borough","Neighborhood","Latitude","Longitude"]
neighborhoods = pd.DataFrame(columns=column_names)

# Fill in the data
for nbh in cells:
    postal_code = nbh[0]
    borough = nbh[1]
    neighborhood = nbh[2]
    if postal_code in np.array(neighborhoods["PostalCode"]):
        # If the postal code already in the dataframe, merge the neighborhood name with the former
        neighborhoods.loc[neighborhoods["PostalCode"] == postal_code,"Neighborhood"] += (", " + neighborhood)
    else:
        neighborhoods = neighborhoods.append({"PostalCode":postal_code,
                                              "Borough":borough,
                                              "Neighborhood":neighborhood,
                                              "Latitude":0.0,
                                              "Longitude":0.0}, ignore_index=True)


In [4]:
# Get Latitude and Longitude
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")

for postal_code in np.array(neighborhoods["PostalCode"]):
    neighborhoods.loc[neighborhoods["PostalCode"] == postal_code,"Latitude"] = df_coordinates.loc[df_coordinates["Postal Code"] == postal_code,"Latitude"].item()
    neighborhoods.loc[neighborhoods["PostalCode"] == postal_code,"Longitude"] = df_coordinates.loc[df_coordinates["Postal Code"] == postal_code,"Longitude"].item()

print(neighborhoods.shape)
neighborhoods.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [5]:
# Get latitude and longitude of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Keep only the boroughs with "Toronto" in the name

In [7]:
neighborhoods_toronto = neighborhoods[["Toronto" in name for name in neighborhoods["Borough"]]]
neighborhoods_toronto.reset_index(drop = True, inplace = True)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Get the top 100 venues in each neighborhood within a radius of 800 meters
#### Radius set to 800m after experiments from 200m to 1km

In [8]:
# Client data for api
CLIENT_ID = 'WJX3QWC3Y0EIDLAX0QHONQ5IDKTPICQ4RZLQ0AESQUWHJYTT'
CLIENT_SECRET = 'AVZGHHSXFT0GS5E1BA5OBT12Q4ZKGKA5JIABZKKO0YQCNM10'
VERSION = '20190731'

In [9]:
# Define the method for getting the venues
def getNearbyVenues(names, latitudes, longitudes, radius=1000, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
# Get venues
toronto_venues = getNearbyVenues(names=neighborhoods_toronto['Neighborhood'],
                                   latitudes=neighborhoods_toronto['Latitude'],
                                   longitudes=neighborhoods_toronto['Longitude'], 
                                  )

# Check the shape and the first items
print(toronto_venues.shape)
toronto_venues.head()

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


### One-hot Coding for all venues

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# group the boroughs
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.000000,0.030000,0.0,0.0,0.0,0.0,0.01,0.01,0.010000,0.02,0.00,0.0,0.000000,0.00,0.020000,0.0,0.020000,0.0,0.0,0.00,0.000000,0.00,0.02,0.0,0.000000,0.01,0.0,0.01,0.020000,0.000000,0.0,0.00,0.0,0.0,0.010000,0.0

### Cluster neighborhood
#### kclusters set to 6 after experiments from 3 to 9

In [12]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 4, 0, 5, 4, 1, 4, 4, 1])

In [13]:
# add clustering labels
neighborhoods_toronto_merged = neighborhoods_toronto
neighborhoods_toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_toronto_merged

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,1,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,1,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,4,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,4,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,1,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_toronto_merged['Latitude'], neighborhoods_toronto_merged['Longitude'], neighborhoods_toronto_merged['Neighborhood'], neighborhoods_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters